# `Fall 2022 MGMT 59000-005`
## **Final Group Project - AUD Ones Out**
<br>
<div style="text-align: justify"> Craigslist offers users access to buy and sell online through free classified ads. When you access the website, it's important to protect your personal email from unscrupulous users who may spam you by offering fraudulent payments for your item. </div> 

### Purpose of this Notebook:
* Text Analytics
* Topic Modelling

<br>

### Version History:
| Author | Description | Execution Duration (mins) |
| --- | --- | --- |
|Vedanti Gulalkari | Text Analytics | 1 |
|Mrinmoy Dalal | Topic Modelling | 15 |

In [47]:
# !pip install contractions
import pandas as pd
import nltk
import numpy as np
# import contractions
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import unicodedata
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb

In [48]:
def weird_replacer(text):
    replacer_dict = {"‚Äô":"'", "/":" ", "-":" ", "‚Äù":"\"", "'":" ", "We've detected that JavaScript is not enabled in your browser. You must enable JavaScript to use craigslist":"", "...":"", "  ":" ", "~":""}
    for key, value in replacer_dict.items():
        text = text.replace(key, value)
    return text

def replace_contractions(text):
    # Replace contractions in string of text
    return contractions.fix(text)

def remove_URL(text):
    # Remove URLs from a sample string 
    text = text.split()
    text = [word for word in text if "http" not in word]
    return " ".join(text)
    
    return re.sub(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', text, flags=re.MULTILINE)

def lowercase(text):
    # Convert everything to lowercase
    return text.lower()

def tokenize(text):
    return nltk.word_tokenize(text)

# Ops on tokenized data

def remove_non_ascii(words):
    # Remove non-ASCII characters from list of tokenized words
    return [unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore') for word in words]

def remove_stopwords_punctuation(words):
    # Remove punctuation from list of tokenized words
    stopwords_list = stopwords.words('english') + list(string.punctuation)
    return [word for word in words if word not in stopwords_list]

def remove_digits(words):
    # Remove digits
    return [word for word in words if word.isalpha()] #not word.isdigit()]

def stem_words(words):
    # Stem words in list of tokenized words
    stemmer = LancasterStemmer()
    return [stemmer.stem(word) for word in words]

def lemmatize_verbs(words):
    # Lemmatize verbs in list of tokenized words
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word, pos='v') for word in words]

def normalize(words):
    words = remove_non_ascii(words)
    words = remove_stopwords_punctuation(words)
    words = remove_digits(words)
    #words = stem_words(words)
    #words = lemmatize_verbs(words)
    desc = " ".join(words)
    return desc

def clean_data(text):
    text = remove_URL(text)
    text = weird_replacer(text)
    # text = replace_contractions(text)    
    text = lowercase(text)
    return tokenize(text)

In [49]:
df = pd.read_excel("CombinedChicago_clean4_classified_draft.xlsx").dropna()
df["Category"] = df["Category"].str.lower()
df["cleaned_desc"] = ""
df["condition"] = ""
df

,Title,Links,Desc,Spam Flag,Category,cleaned_desc,condition
0,Home Theater,https://chicago.craigslist.org/chc/ele/d/jolie...,- electronics - city of chicago for sale elec...,0.0,speakers and microphones,,
1,ITE Power Supply,https://chicago.craigslist.org/sox/ele/d/westc...,- electronics - city of chicago for sale elec...,0.0,computer equipment,,
2,JBL Aquarius,https://chicago.craigslist.org/chc/ele/d/chica...,- electronics - city of chicago for sale elec...,0.0,speakers and microphones,,
3,Ar 3a Woofer,https://chicago.craigslist.org/chc/ele/d/chica...,- electronics - city of chicago for sale elec...,0.0,speakers and microphones,,
4,TASCAM DR-40,https://chicago.craigslist.org/chc/ele/d/chica...,- electronics - city of chicago for sale elec...,0.0,speakers and microphones,,
...,...,...,...,...,...,...,...
2956,HP Procurve ZL J9643A 288x POE+ Gigabit switch...,https://chicago.craigslist.org/chc/ele/d/chica...,ZL J9643A 288x POE+ Gigabit switch ports (cisc...,0.0,networking equipment,,
2957,Speakers Infinity SM120,https://chicago.craigslist.org/nwi/ele/d/crown...,ZL J9643A 288x POE+ Gigabit switch ports (cisc...,0.0,networking equipment,,
2958,Chauvet DJ EZpar T6 USB Battery Tri-LED Wash L...,https://chicago.craigslist.org/chc/ele/d/chauv...,Zpar T6 USB Battery Tri-LED Wash Lights (set o...,0.0,battery,,
2959,Chauvet DJ EZpar T6 USB Battery Tri-LED Wash L...,https://chicago.craigslist.org/chc/ele/d/chauv...,Zpar T6 USB Battery Tri-LED Wash Lights (set o...,0.0,battery,,


In [50]:
for idx, row in df.iterrows():
    desc = row["Desc"]

    test = ""

    condition_index = desc.find("condition: ")
    if (condition_index != -1):
        test = desc[condition_index:]
        test = test.split(" ")
        test = test[1]
        if test == "like":
            test = "like new"
    else:
        test = "unavailable"
    df.loc[idx, "condition"] = test
        
    desc = desc[desc.find(" Post ")+6:]
    end_index = desc.find("it's ok to contact this poster with services or other")
    if end_index != 1:
        desc = desc[:end_index]

    desc = row["Title"] + " " + desc

    desc = clean_data(desc)
    desc = normalize(desc)
    df.loc[idx, "cleaned_desc"] = desc
    print(idx, desc)


0 home theater brand new box high quality surround theater hd sound works projectors much hit take best offer
1 ite power supply let use nest product homekit works perfectly
2 jbl aquarius vintage speakers local pickup cash
3 ar woofer selling ar woofer pairs ar many years ago purchased woofer spare speaker surround deteriorated need replacing magnet dated unable read thank
4 tascam dr older dr purchased several years back internal mic works try switch external mic turn still good interviewing field recordings music recording
5 sony bravia television adjustable wall mount brand new never opened first come first serve
6 big speakers big loud speakers american acoustic labs mtx well built solid cases feature latch heavy duty covers easy transportation storage sound good incredible highs banks tweeters thunderously loud lows dual woofers lower drivers new replacements made cerwin vega
7 nintendo dsi sale nintendo dsi hand held game system model appears fully functional rough spots around 

In [52]:
df_subset = df.iloc[:,[0,3,4,5,6]].drop_duplicates()
df_subset.loc[df_subset["Category"] == "calculator", "Category"] = "miscellaneous"
df_subset.loc[df_subset["Category"] == "light", "Category"] = "miscellaneous"

df_subset.loc[df_subset["Category"] == "watch", "Category"] = "phone/tablet/watch"

df_subset.loc[df_subset["Category"] == "phone", "Category"] = "phone/tablet/watch"

df_subset.loc[df_subset["Category"] == "tablet", "Category"] = "phone/tablet/watch"

In [53]:
df_subset.Category = pd.Categorical(df_subset.Category)
df_subset["num_cat"] = pd.factorize(df_subset["Category"])[0]
df_subset

,Title,Spam Flag,Category,cleaned_desc,condition,num_cat
0,Home Theater,0.0,speakers and microphones,home theater brand new box high quality surrou...,new,0
1,ITE Power Supply,0.0,computer equipment,ite power supply let use nest product homekit ...,excellent,1
2,JBL Aquarius,0.0,speakers and microphones,jbl aquarius vintage speakers local pickup cash,good,0
3,Ar 3a Woofer,0.0,speakers and microphones,ar woofer selling ar woofer pairs ar many year...,good,0
4,TASCAM DR-40,0.0,speakers and microphones,tascam dr older dr purchased several years bac...,unavailable,0
...,...,...,...,...,...,...
2955,Sony remote,0.0,networking equipment,sony remote hp procurve zl gigabit switch port...,excellent,12
2956,HP Procurve ZL J9643A 288x POE+ Gigabit switch...,0.0,networking equipment,hp procurve zl gigabit switch ports cisco juni...,unavailable,12
2957,Speakers Infinity SM120,0.0,networking equipment,speakers infinity hp procurve zl gigabit switc...,unavailable,12
2958,Chauvet DJ EZpar T6 USB Battery Tri-LED Wash L...,0.0,battery,chauvet dj ezpar usb battery tri led wash ligh...,like new,11


In [103]:
train_x, test_x, train_y, test_y = train_test_split(df_subset.cleaned_desc, df_subset.num_cat, test_size=0.3, random_state=69)

def tk(doc):
    return doc

vec = TfidfVectorizer(analyzer='word', tokenizer=tk, preprocessor=tk, token_pattern=None,
                      min_df=2, ngram_range=(1,3))

vec.fit(train_x)
train_x = vec.transform(train_x)
test_x = vec.transform(test_x)

In [104]:
model = []

from sklearn.naive_bayes import BernoulliNB

naive_bayes = BernoulliNB()
naive_bayes.fit(train_x, train_y)
y_pred_naive_bayes = naive_bayes.predict(test_x)

print(classification_report(test_y, y_pred_naive_bayes))

              precision    recall  f1-score   support

           0       0.69      0.60      0.64       181
           1       0.62      0.12      0.20        42
           2       0.44      0.74      0.55        78
           3       0.00      0.00      0.00        32
           4       0.33      0.50      0.40        18
           5       0.19      0.73      0.30        52
           6       0.25      0.28      0.27       115
           7       0.38      0.08      0.13        38
           8       0.66      0.66      0.66        50
           9       0.56      0.41      0.47        56
          10       0.31      0.24      0.27        46
          11       0.00      0.00      0.00        14
          12       0.78      0.60      0.68        30
          13       0.47      0.25      0.33        36
          14       1.00      0.06      0.11        18
          15       0.00      0.00      0.00         4
          16       0.29      0.33      0.31        12
          17       0.17    

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

In [105]:
from sklearn.ensemble import RandomForestClassifier
rand_forest_model = RandomForestClassifier(n_estimators=500, max_depth=6, bootstrap=True, random_state=0,class_weight = 'balanced')  # with 10 trees
rand_forest_model.fit(train_x, train_y)
y_pred_rand_forest = rand_forest_model.predict(test_x)
print("Random Forest:::")
print(classification_report(test_y, y_pred_rand_forest))

Random Forest:::
              precision    recall  f1-score   support

           0       0.80      0.54      0.65       181
           1       0.33      0.17      0.22        42
           2       0.65      0.77      0.71        78
           3       0.50      0.59      0.54        32
           4       0.58      0.83      0.68        18
           5       0.53      0.58      0.55        52
           6       0.20      0.02      0.03       115
           7       0.34      0.32      0.33        38
           8       0.69      0.74      0.71        50
           9       0.62      0.59      0.61        56
          10       0.49      0.76      0.59        46
          11       0.31      0.57      0.40        14
          12       0.49      0.70      0.58        30
          13       0.41      0.67      0.51        36
          14       0.38      0.78      0.51        18
          15       0.13      0.75      0.22         4
          16       0.61      0.92      0.73        12
          

In [106]:
from sklearn.svm import LinearSVC
SVMmodel = LinearSVC(random_state = 0, penalty = 'l1', dual = False)
SVMmodel.fit(train_x, train_y)
y_pred_SVM = SVMmodel.predict(test_x)
model.append("SVM")
print("SVM:::")
print(classification_report(test_y, y_pred_SVM))

SVM:::
              precision    recall  f1-score   support

           0       0.71      0.87      0.78       181
           1       0.74      0.48      0.58        42
           2       0.72      0.81      0.76        78
           3       0.73      0.50      0.59        32
           4       1.00      0.83      0.91        18
           5       0.74      0.77      0.75        52
           6       0.51      0.60      0.55       115
           7       0.48      0.26      0.34        38
           8       0.82      0.82      0.82        50
           9       0.69      0.71      0.70        56
          10       0.82      0.59      0.68        46
          11       0.54      0.50      0.52        14
          12       0.73      0.73      0.73        30
          13       0.68      0.72      0.70        36
          14       0.67      0.56      0.61        18
          15       0.00      0.00      0.00         4
          16       0.91      0.83      0.87        12
          17       0

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [107]:
# !pip install xgboost

In [108]:
#xgboost
xgb_classifier = xgb.XGBClassifier()
xgb_classifier.fit(train_x,train_y)
predictions = xgb_classifier.predict(test_x)
print(classification_report(test_y, predictions))

              precision    recall  f1-score   support

           0       0.74      0.81      0.77       181
           1       0.52      0.26      0.35        42
           2       0.71      0.78      0.74        78
           3       0.70      0.50      0.58        32
           4       0.93      0.78      0.85        18
           5       0.76      0.75      0.76        52
           6       0.41      0.51      0.46       115
           7       0.42      0.29      0.34        38
           8       0.79      0.82      0.80        50
           9       0.73      0.73      0.73        56
          10       0.85      0.74      0.79        46
          11       0.50      0.50      0.50        14
          12       0.67      0.53      0.59        30
          13       0.64      0.69      0.67        36
          14       0.67      0.44      0.53        18
          15       0.33      0.75      0.46         4
          16       0.77      0.83      0.80        12
          17       0.38    

In [129]:
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier
#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=7)
#Train the model using the training sets
knn.fit(train_x, train_y)
#Predict the response for test dataset
y_pred = knn.predict(test_x)
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       0.41      0.88      0.56       181
           1       0.62      0.24      0.34        42
           2       0.75      0.64      0.69        78
           3       0.76      0.41      0.53        32
           4       0.71      0.83      0.77        18
           5       0.65      0.50      0.57        52
           6       0.26      0.33      0.29       115
           7       0.78      0.18      0.30        38
           8       0.97      0.56      0.71        50
           9       0.64      0.50      0.56        56
          10       0.75      0.13      0.22        46
          11       0.60      0.21      0.32        14
          12       0.71      0.50      0.59        30
          13       0.76      0.36      0.49        36
          14       0.50      0.11      0.18        18
          15       0.00      0.00      0.00         4
          16       0.89      0.67      0.76        12
          17       0.00    

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

In [127]:
#Nueral Network
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(18,18,18), activation='relu', solver='adam', max_iter=9999)
mlp.fit(train_x,train_y)

predict_train = mlp.predict(train_x)
predict_test = mlp.predict(test_x)
print(classification_report(test_y, predict_test))

              precision    recall  f1-score   support

           0       0.62      0.65      0.63       181
           1       0.41      0.29      0.34        42
           2       0.70      0.71      0.70        78
           3       0.52      0.34      0.42        32
           4       0.48      0.61      0.54        18
           5       0.57      0.62      0.59        52
           6       0.33      0.37      0.35       115
           7       0.24      0.24      0.24        38
           8       0.61      0.62      0.61        50
           9       0.56      0.55      0.56        56
          10       0.34      0.28      0.31        46
          11       0.43      0.21      0.29        14
          12       0.52      0.57      0.54        30
          13       0.53      0.58      0.55        36
          14       0.50      0.17      0.25        18
          15       0.17      0.50      0.25         4
          16       0.67      0.50      0.57        12
          17       0.23    

In [109]:
# lda = LatentDirichletAllocation(n_components=10, max_iter=200, random_state = 999).fit(x)
# topic_prob = lda.transform(x)

In [110]:
# topix = []
# for topic_idx, topic in enumerate(lda.components_):
#     print("Topic %d:" %(topic_idx+1), end = " ")
#     print(", ".join([terms[i] for i in topic.argsort()[:-5:-1]]))
#     topix.append(", ".join([terms[i] for i in topic.argsort()[:-5:-1]]))